In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from category_encoders import TargetEncoder, OneHotEncoder, CountEncoder, OrdinalEncoder
import random
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error as rmse
from statistics import stdev

In [ ]:
allegro = pd.read_csv('allegro-api-transactions.csv')

allegro.head()

In [ ]:
allegro.info()

## Zadanie 1

Wykonaj target encoding dla zmiennej it_location. Czy i jakie są przewagi target encoding nad one-hot? Jako target traktujemy kolumnę price (będzie to więc zadanie regresji).

Zastosuj trzy metody encodingu (one-hot + "dwie nowe") dla kolumny main_category. "Nowe metody" proszę wybrać spośród wymienionych na stronie https://contrib.scikit-learn.org/category_encoders/. W przypadku, gdy użyta metoda nie działa proszę o stosowną adnotację. Opisz wyniki.

Zwizalizuj wynik oraz wyjaśnij czym się różnią sposoby kodowania (czemu to działa).

In [ ]:
location = allegro[['it_location']].groupby(['it_location']).size().sort_values(ascending=False).reset_index()
location.columns = ['location', 'count']

location.loc[location['location'].apply(lambda x:'warszawa' in x.lower())]

Warszawa, WARSZAWA i warszawa to zupełnie to samo, można ujednolicić, trzeba się zastanowić nad innymi nazwami. 

Tak samo jest z innymi miastami, żeby ujednolicić najłatwiej przekształcić wszystkie nazwy tak, żeby skłądały się tylko z małych liter, nie będzie wtedy problemu z wieloczłonowymi nazwami.

In [ ]:
allegro['it_location'] = allegro['it_location'].apply(lambda x: x.lower())

In [ ]:
allegro['it_location'].value_counts()

In [ ]:
target_encoder = TargetEncoder()

allegro['target_encoding_location'] =  target_encoder.fit_transform(allegro['it_location'], allegro['price'])

In [ ]:
allegro.head()

Pojawiła się nowa kolumna - 'target_encoding_location', zakodowana wartość to średnia z wartośći kolumny 'price', dla danej wartości 'it_location'.

In [ ]:
try:
    OneHotEncoder(use_cat_names=True, cols = ['it_location']).fit_transform(allegro.it_location)
except MemoryError as error:
    print('Brakuje pamieci, ', error)

OneHotEncoding potrzbuje dużo dodatkowej pamięci do zakodowania zmiennej 'it_location', chcąc dołączyć zakodowaną macierz do ramki danych potrzebował by jeszcze znacznie więcej zasobów - to spory minus, kolejna pamięć byłaby potrzeba na wykonywanie operacji na nowo powstałej ramce. 

W tym przypadku TargetEncoder sprawdza się znacznie lepiej, lecz on też ma swoje wady. Kodując za pomocą średniej stwarza możliwość porównywania zakodowanych wartości, co może być mylące.

### OneHotEncoder

Kodowanie kolumny 'main_category'

In [ ]:
OneHotEncoder(cols = ['main_category'], use_cat_names=True).fit_transform(allegro.main_category)

### CountEncoder

Każdej wartości z kolumny 'main_categoty' przyporządkowuje liczbę jej wystąpień.

In [ ]:
CountEncoder(cols=['main_category']).fit_transform(allegro.main_category)

### OrdinalEncoder

Tworzy kolumne intów, każdej unikatowj wartości z 'main_category' przyporządkowuje kolejną wartość naturalną.

In [ ]:
OrdinalEncoder(cols=['main_category']).fit_transform(allegro.main_category)

Tak jak poprzednio, OneHotEncoding potrzebuje najwięcej pamięci i dodaje do ramki 27 nowych kolumn. CountEncoder i OrdinalEncoder zachowują się podobnie do TargetEncoder, Count zamiast średniej liczy liczbę wystąpień, da się porównywać zakodowane zmienne, co nie jest dobrą cechą. Ordinal dopasowuje kolejno dobrane liczby naturalne, co za tym idzie nie mają one nic wspólengo z kodowaną zmienną, również da się je porównywać.

## Zadanie 2

W tej części zadania traktujemy zmienną price nie jak target a zmienną objaśniającą. Zbiór danych ograniczamy do zmiennych numerycznych tj. price, it_seller_rating i it_quantity.

Proszę losowo usunąć 10% wartości ze zmiennej it_seller_rating i je uzupełnić z użyciem jednego z automatycznych narzędzi: Nearest neighbors imputation lub Multivariate feature imputation (https://scikit-learn.org/stable/modules/impute.html).
Następnie należy porównać wartości imputowane z oryginalnymi (polecam miarę RMSE). Eksperyment powtórzyć 10 razy i zobaczyć jakie będzie odchylenie standardowe wyniku. Następnie zrobić analogiczną analizę gdy oprócz losowego usuwania 10% wartości z kolumny it_seller_rating usuniemy także losowo 10% ze zmiennej it_quantity. (w przypadku problemów wydajnościowych proszę ograniczyć liczbę rekordów).

Opisać wnioski z analizy jakości imputacji i umieścić podsumowujący wykres.

In [ ]:
allegro_num = allegro[['price', 'it_seller_rating', 'it_quantity']].sample(10000).reset_index(drop = True)

#ognaiczyłem dane ze względu na problemy z wydajnością

In [ ]:
print(allegro_num.shape)
allegro_num.head()

In [ ]:
allegro_num.info()

In [ ]:
indexes = allegro_num.sample(int(len(allegro_num) * 0.1)).index

lost_data = allegro_num.copy(deep = True)

lost_data.loc[indexes,'it_seller_rating'] = np.nan

In [ ]:
lost_data.info()

In [ ]:
imputer = KNNImputer(n_neighbors = 3,weights = 'uniform').fit_transform(lost_data)

filled_data = pd.DataFrame(imputer)

filled_data.columns = ['price', 'it_seller_rating', 'it_quantity']

In [ ]:
filled_data.head()

In [ ]:
filled_data.info()

In [ ]:
rmse(allegro_num.it_seller_rating, filled_data.it_seller_rating, squared=False)

In [ ]:
vals = []

for i in range(10):
    indexes = allegro_num.sample(int(len(allegro_num) * 0.1)).index
    lost_data = allegro_num.copy(deep = True)
    lost_data.loc[indexes,'it_seller_rating'] = np.nan 
    imputer = KNNImputer(n_neighbors = 3,weights = 'uniform').fit_transform(lost_data)
    filled_data = pd.DataFrame(imputer)
    filled_data.columns = ['price', 'it_seller_rating', 'it_quantity']
    
    vals.append(rmse(allegro_num.it_seller_rating, filled_data.it_seller_rating, squared=False))
    
stdev(vals)
    

In [ ]:
allegro_num = allegro_num.sample(5000)
#musiałem jeszcze bardziej ograniczyć dane, żeby użyć imputera dla dwóch kolumn

vals2 = []

for i in range(10):
    indexes = allegro_num.sample(int(len(allegro_num) * 0.1)).index
    lost_data = allegro_num.copy(deep = True)
    lost_data.loc[indexes,'it_seller_rating'] = np.nan 
    indexes = allegro_num.sample(int(len(allegro_num) * 0.1)).index
    lost_data.loc[indexes,'it_quantity'] = np.nan 
    
    imputer = KNNImputer(n_neighbors = 3,weights = 'uniform').fit_transform(lost_data)
    filled_data = pd.DataFrame(imputer)
    filled_data.columns = ['price', 'it_seller_rating', 'it_quantity']
    
    vals2.append(rmse(allegro_num.it_seller_rating, filled_data.it_seller_rating, squared=False))
    
stdev(vals2)

In [ ]:
dict = {'missing_cols': np.concatenate([['one_col']*10, ['two_cols']*10]), 
     'rmse': np.concatenate([vals, vals2])}
df = pd.DataFrame(data=dict)

fig, ax = plt.subplots(figsize=(6, 4))
sns.boxplot(data = df, x = 'missing_cols', y = 'rmse', ax = ax)
plt.title('Comprasion for RMSE values for one and two missing columns')
plt.show()

Wartości RMSE są bardzo duże, czyli imputacja działa słabo. 

Uzupełnienie brakujących wartości w dwóch kolumnach daje gorsze wyniki niż w jednej.